In [1]:
import os
import urllib

import pandas as pd

from cell_type_naming import nice_to_weirds, weird_to_nice

In [2]:
experiment_name = "website_inferred_refsample"
experiment_uri = os.path.join("gs://liulab/csx_experiments/varying_simple_parameters", experiment_name)

experiment_uri

'gs://liulab/csx_experiments/varying_simple_parameters/website_inferred_refsample'

In [3]:
!gsutil --quiet rm -r $experiment_uri

In [4]:
mixture_source = "gs://liulab/csx_example_files/Single_Cell_RNA-Seq_Melanoma_SuppFig_3b-d/mixture_melanoma_Tirosh_SuppFig_3b-d.txt"
# mixture_filename = os.path.basename(mixture_source)
mixture_filename = "mymixture.txt"
mixture_target = os.path.join(experiment_uri, "in", mixture_filename)

!gsutil cp $mixture_source $mixture_target

Copying gs://liulab/csx_example_files/Single_Cell_RNA-Seq_Melanoma_SuppFig_3b-d/mixture_melanoma_Tirosh_SuppFig_3b-d.txt [Content-Type=text/plain]...
/ [1 files][  6.0 MiB/  6.0 MiB]                                                
Operation completed over 1 objects/6.0 MiB.                                      


In [5]:
%%bash

# refsample
gsutil cp \
    gs://liulab/downloaded_manually/refsamples_from_csx_website/CIBERSORTx_Job4_myrefsample_inferred_refsample.txt \
    gs://liulab/csx_experiments/varying_simple_parameters/website_inferred_refsample/in/Job4_inferred_refsample.txt

# phenoclasses
gsutil cp \
    gs://liulab/downloaded_manually/refsamples_from_csx_website/CIBERSORTx_Job4_myrefsample_inferred_phenoclasses.txt \
    gs://liulab/csx_experiments/varying_simple_parameters/website_inferred_refsample/in/Job4_inferred_phenoclasses.txt


Copying gs://liulab/downloaded_manually/refsamples_from_csx_website/CIBERSORTx_Job4_myrefsample_inferred_refsample.txt [Content-Type=text/plain]...
/ [1 files][  4.9 MiB/  4.9 MiB]                                                
Operation completed over 1 objects/4.9 MiB.                                      
Copying gs://liulab/downloaded_manually/refsamples_from_csx_website/CIBERSORTx_Job4_myrefsample_inferred_phenoclasses.txt [Content-Type=text/plain]...
/ [1 files][  725.0 B/  725.0 B]                                                
Operation completed over 1 objects/725.0 B.                                      


In [6]:
experiment_path = "/mnt/buckets/liulab" + urllib.parse.urlparse(experiment_uri).path

In [7]:
!tree -h $experiment_path

/mnt/buckets/liulab/csx_experiments/varying_simple_parameters/website_inferred_refsample
└── [   0]  in
    ├── [ 725]  Job4_inferred_phenoclasses.txt
    ├── [4.9M]  Job4_inferred_refsample.txt
    └── [6.0M]  mymixture.txt

1 directory, 3 files


In [8]:
command = f'''docker run \
    --rm \
    -v {experiment_path}/in:/src/data \
    -v {experiment_path}:/src/outdir \
    --user "$(id -u):$(id -g)" \
    cibersortx/fractions:latest \
    --username lyronctk@stanford.edu \
    --token dfeba2c8b9d61daebee5fa87026b8e56 \
    --single_cell FALSE \
    --refsample Job4_inferred_refsample.txt \
    --phenoclasses Job4_inferred_phenoclasses.txt \
    --mixture {mixture_filename} \
    --rmbatchBmode TRUE \
    --verbose TRUE
'''.replace("    ", " \\\n    ")
print(command)

docker run \
     --rm \
     -v /mnt/buckets/liulab/csx_experiments/varying_simple_parameters/website_inferred_refsample/in:/src/data \
     -v /mnt/buckets/liulab/csx_experiments/varying_simple_parameters/website_inferred_refsample:/src/outdir \
     --user "$(id -u):$(id -g)" \
     cibersortx/fractions:latest \
     --username lyronctk@stanford.edu \
     --token dfeba2c8b9d61daebee5fa87026b8e56 \
     --single_cell FALSE \
     --refsample Job4_inferred_refsample.txt \
     --phenoclasses Job4_inferred_phenoclasses.txt \
     --mixture mymixture.txt \
     --rmbatchBmode TRUE \
     --verbose TRUE



In [9]:
!{command}

>Running CIBERSORTxFractions...
>[Options] username: lyronctk@stanford.edu
>[Options] token: dfeba2c8b9d61daebee5fa87026b8e56
>[Options] single_cell: FALSE
>[Options] refsample: Job4_inferred_refsample.txt
>[Options] phenoclasses: Job4_inferred_phenoclasses.txt
>[Options] mixture: mymixture.txt
>[Options] rmbatchBmode: TRUE
>[Options] verbose: TRUE
>==========Signature Matrix Settings============
>Pure samples file: /src/data/Job4_inferred_refsample.txt
>Phenotype classes file: /src/data/Job4_inferred_phenoclasses.txt
>Quantile normalization of input samples is disabled
>===============================================
>C++14
>Loaded 40 columns and 23684 rows
>=============Make Signature Matrix=============
>Maximum condition number (kappa):999
>71036 DEG expressed across 8 samples were identified
>Group size: 50, Kappa: 9.44702958017558
>Best kappa: 9.44702958017558 
>SVD UpperRightDiag: 20843.6
>SVD LowerRightDiag: 2206.37
>=============CIBERSORTx Settings===============
>Mixture file

In [10]:
!tree -h $experiment_path

/mnt/buckets/liulab/csx_experiments/varying_simple_parameters/website_inferred_refsample
├── [ 25K]  CIBERSORTx_Job4_inferred_phenoclasses.CIBERSORTx_Job4_inferred_refsample.bm.K999.pdf
├── [ 59K]  CIBERSORTx_Job4_inferred_phenoclasses.CIBERSORTx_Job4_inferred_refsample.bm.K999.txt
├── [4.3K]  CIBERSORTx_Results.txt
├── [3.8M]  CIBERSORTx_cell_type_sourceGEP.txt
└── [   0]  in
    ├── [ 725]  Job4_inferred_phenoclasses.txt
    ├── [4.9M]  Job4_inferred_refsample.txt
    └── [6.0M]  mymixture.txt

1 directory, 7 files


In [11]:
fractions_csx = pd.read_csv(
    os.path.join(experiment_uri, "CIBERSORTx_Adjusted.txt"),
    sep="\t",
    index_col=0
).rename_axis(index="mixture_id").rename(columns=weird_to_nice).iloc[:, :-3]

fractions_csx.sample(5)

FileNotFoundError: 

In [ ]:
signature_matrix = pd.read_csv(
    os.path.join(experiment_uri, "CIBERSORTx_myscrefsample_inferred_phenoclasses.CIBERSORTx_myscrefsample_inferred_refsample.bm.K999.txt"),
    sep="\t",
    index_col=0
).rename_axis(index="gene").rename(columns=weird_to_nice)

signature_matrix.sample(5)

In [ ]:
fractions_truth = pd.read_csv(
    "gs://liulab/csx_example_files/groundtruth_Melanoma_Tirosh_et_al_SuppFig3b-d.txt",
    sep="\t",
    index_col=0,
).rename_axis(index="mixture_id").rename(columns=weird_to_nice)

fractions_truth.sample(5)

In [ ]:
import plotly.io
plotly.io.renderers.default = "jupyterlab+png"

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
fig = px.bar(
    fractions_truth.sort_values(by="Malignant"),  #.astype(index={"mixture_id": str}),
    x=fractions_truth.index.astype(str),
    y=fractions_truth.columns,
    title="True fractions of each mixture"
)

fig.show()

In [ ]:
fig = px.bar(
    fractions_csx,  #.astype(index={"mixture_id": str}),
    x=fractions_csx.index.astype(str),
    y=fractions_csx.columns,
    title="Predicted fractions of each mixture"
)

fig.show()